In [90]:
# Install dependencies
!pip install -q litellm mlflow langchain langchain-community faiss-cpu sentence-transformers pypdf ragas langchain-groq groq datasets langchain_huggingface polars

In [46]:
# --- Imports ---
import os
# import mlflow
import litellm
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision, FactualCorrectness
from ragas.llms import LangchainLLMWrapper
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings as RagasHFEmbeddings
from datasets import Dataset
from google.colab import userdata

In [47]:
# --- Environment Setup ---
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY_")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
# mlflow.set_experiment("coal_mining_self_rag")
# mlflow.litellm.autolog()

2025/07/13 09:50:53 INFO mlflow.tracking.fluent: Experiment with name 'coal_mining_self_rag' does not exist. Creating a new experiment.


In [48]:
# --- Load PDFs ---
pdf_folder = "/content/IncidentManuals"
loaders = [PyPDFLoader(os.path.join(pdf_folder, fn)) for fn in os.listdir(pdf_folder) if fn.endswith(".pdf")]
documents = []
for loader in loaders:
    documents.extend(loader.load())

In [49]:
# --- Chunking ---
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)

In [50]:
# --- Embedding & VectorStore ---
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(chunks, embedding)
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"lambda_mult": 0.5, "k": 5})

In [89]:
import polars as pl
# Create a list of dictionaries, one for each chunk
dta = []
for i, chunk in enumerate(chunks):
    dta.append({
        "chunk_id": i,
        "content": chunk.page_content,
        "source": chunk.metadata.get("source"),
        "page": chunk.metadata.get("page"),
        # You could also potentially get the embedding if needed,
        # but FAISS doesn't make it easy to retrieve embeddings by index directly.
        # "embedding": embedding.embed_query(chunk.page_content) # This would be slow
    })
# Create a Polars DataFrame
df = pl.DataFrame(dta)
# Display the DataFrame
df.head()

chunk_id,content,source,page
i64,str,str,i64
0,"""Combine work of multiple colla…","""/content/IncidentManuals/testP…",0
1,"""functionality. The aim is to f…","""/content/IncidentManuals/testP…",0
2,"""S.O.L.I.D. Python""","""/content/IncidentManuals/solid…",0
3,"""Alea Soluciones Bifer Team""","""/content/IncidentManuals/solid…",1
4,"""@eferro @pasku1 @apa42 @nesto…","""/content/IncidentManuals/solid…",2


In [51]:
# --- Prompts ---
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a python Software Progremmer. Use the context below to answer questions accurately."),
    ("user", "Context:\n{context}\n\nQuestion: {question}")
])
editor_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a System Design and Python Expert. Refine the answer for accuracy and clarity."),
    ("user", "Draft:\n{draft}")
])

In [52]:
# --- Internal Memory ---
memory = FAISS.from_texts(["init"], embedding)

def retrieve_self(query):
    memory_results = memory.similarity_search(query)
    return memory_results if memory_results else retriever.get_relevant_documents(query)

In [53]:
# --- Self-RAG Pipeline ---
def generate(query):
    context_docs = retrieve_self(query)
    context = "\n\n".join([doc.page_content for doc in context_docs])

    draft = litellm.completion(
        model="openai/gpt-4o-mini",
        messages=[{"role": "user", "content": rag_prompt.format(context=context, question=query)}]
    )
    draft_content = draft['choices'][0]['message']['content']

    refined = litellm.completion(
        model="groq/llama3-70b-8192",
        messages=[{"role": "user", "content": editor_prompt.format(draft=draft_content)}]
    )
    refined_content = refined['choices'][0]['message']['content']
    memory.add_texts([refined_content])

    return refined_content, context

In [66]:
# --- Python SOLID Principles Q&A Dataset ---
questions = [
    "What is the Single Responsibility Principle and how is it applied in Python?",
    "Explain the Open/Closed Principle with an example in Python.",
    "Why is the Liskov Substitution Principle important in object-oriented design?",
    "How does the Interface Segregation Principle improve software flexibility?",
    "What is the Dependency Inversion Principle and how is it implemented in Python?"
]

references = [
    "The Single Responsibility Principle states that a class should have only one reason to change. In Python, this can be implemented by separating business logic from data access logic into different classes.",
    "The Open/Closed Principle means software entities should be open for extension but closed for modification. In Python, using abstract base classes and inheritance allows you to add new behavior without altering existing code.",
    "The Liskov Substitution Principle ensures that subclasses can be used in place of base classes without breaking functionality. Violations can lead to unexpected behavior in polymorphic code.",
    "The Interface Segregation Principle advocates for small, specific interfaces rather than large, general ones. In Python, this can be achieved through multiple inheritance or protocols to keep classes decoupled.",
    "The Dependency Inversion Principle suggests high-level modules should not depend on low-level modules, but both should depend on abstractions. In Python, this can be enforced through dependency injection and abstract base classes."
]

In [67]:
from ragas.metrics import Faithfulness, AnswerRelevancy, ContextRecall, ContextPrecision, FactualCorrectness

metrics = [
    Faithfulness(),
    AnswerRelevancy(),
    ContextRecall(),
    ContextPrecision(),
    FactualCorrectness()
]

In [68]:
# --- Generate Answers ---
data = {"question": [], "response": [], "retrieved_contexts": [], "reference": []}
for q, ref in zip(questions, references):
    ans, ctx = generate(q)
    data["question"].append(q)
    data["response"].append(ans)
    # Ensure context is stored as a list of strings for retrieved_contexts
    data["retrieved_contexts"].append([ctx])
    data["reference"].append(ref)

# --- Create HuggingFace Dataset ---
ds = Dataset.from_dict(data)

In [69]:
# --- RAGAS Evaluation ---
llm = LangchainLLMWrapper(
    ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="llama3-8b-8192", temperature=0.0)
)
ragas_emb = RagasHFEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [70]:
result = evaluate(
    ds,
    metrics=metrics,
    embeddings=ragas_emb
)

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [71]:
result.to_pandas().head()

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,factual_correctness(mode=f1)
0,What is the Single Responsibility Principle an...,[init\n\nHere is a refined draft:\n\nThe prima...,"Here's a refined version of your draft, ensuri...",The Single Responsibility Principle states tha...,0.000000,0.994962,0.0,0.0,0.19
1,Explain the Open/Closed Principle with an exam...,"[Here's a refined version of your draft, ensur...",Here's a refined version of the draft for accu...,The Open/Closed Principle means software entit...,0.000000,0.785956,0.0,0.0,0.84
2,Why is the Liskov Substitution Principle impor...,[Here's a refined version of the draft for acc...,Here is a refined version of the draft with im...,The Liskov Substitution Principle ensures that...,0.000000,0.978356,1.0,0.0,0.56
3,How does the Interface Segregation Principle i...,[Here's a refined version of the draft for acc...,Here's a refined draft:\n\nThe Interface Segre...,The Interface Segregation Principle advocates ...,0.000000,0.970183,0.5,0.0,0.74
4,What is the Dependency Inversion Principle and...,[Here's a refined version of the draft for acc...,Here is a refined version of your draft for ac...,The Dependency Inversion Principle suggests hi...,0.970588,0.991885,0.0,0.0,0.81


In [72]:
!pip install -q pyngrok

In [84]:
import os
import getpass
from pyngrok import ngrok, conf
from google.colab import userdata
os.environ["NGROK"]=userdata.get('NGROK_TOKEN')

In [87]:
# Set up ngrok tunnel for MLflow UI
conf.get_default().auth_token = os.environ["NGROK"]
ngrok.kill()
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://f90aeb3a11f1.ngrok-free.app


In [88]:
!mlflow ui

[2025-07-13 10:25:34 +0000] [34805] [INFO] Starting gunicorn 23.0.0
[2025-07-13 10:25:34 +0000] [34805] [INFO] Listening at: http://127.0.0.1:5000 (34805)
[2025-07-13 10:25:34 +0000] [34805] [INFO] Using worker: sync
[2025-07-13 10:25:34 +0000] [34806] [INFO] Booting worker with pid: 34806
[2025-07-13 10:25:34 +0000] [34807] [INFO] Booting worker with pid: 34807
[2025-07-13 10:25:34 +0000] [34808] [INFO] Booting worker with pid: 34808
[2025-07-13 10:25:34 +0000] [34809] [INFO] Booting worker with pid: 34809
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/mlflow/store/tracking/file_store.py", line 356, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/mlflow/store/tracking/file_store.py", line 454, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

WARNI [pyngrok.process.ngrok] t=2025-07-13T10:26:23+0000 lvl=warn msg="Stopping forwarder" name=http-5000-6e4205bf-1fa6-49b4-aa73-7e450668a7b5 acceptErr="failed to accept connection: Listener closed"
WARNI [pyngrok.process.ngrok] t=2025-07-13T10:26:23+0000 lvl=warn msg="Error restarting forwarder" name=http-5000-6e4205bf-1fa6-49b4-aa73-7e450668a7b5 err="failed to start tunnel: session closed"


[2025-07-13 10:26:23 +0000] [34805] [INFO] Handling signal: int

Aborted!
[2025-07-13 10:26:23 +0000] [34807] [INFO] Worker exiting (pid: 34807)
[2025-07-13 10:26:23 +0000] [34808] [INFO] Worker exiting (pid: 34808)
[2025-07-13 10:26:23 +0000] [34806] [INFO] Worker exiting (pid: 34806)
[2025-07-13 10:26:23 +0000] [34809] [INFO] Worker exiting (pid: 34809)
[2025-07-13 10:26:24 +0000] [34805] [INFO] Shutting down: Master
